In [1]:
import pickle
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [2]:
path="../"

In [3]:
train = pd.concat(objs=[pd.read_csv(path+'ddi2013-type/train.tsv', sep='\t'),
                        pd.read_csv(path+'ddi2013-type/DDI_sinonimi.csv')],
                  ignore_index=True).sample(frac=1)
dev = pd.read_csv(path+'ddi2013-type/dev.tsv', sep='\t')
test_org = pd.read_csv(path+'ddi2013-type/test.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"ddi2013-type/DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"ddi2013-type/DDI_embedding_test.csv")

In [4]:
test = pd.concat(objs=[pd.read_csv(path+'ddi2013-type/DDI_sinonimi_test.csv'), test_org],
                 ignore_index=True)

In [5]:
with open(path+"word2index.pkl", 'rb') as output:
    w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
    embedding_matrix = pickle.load(output)

In [6]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

In [7]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train.label.to_numpy().reshape(-1, 1))

sentences_train = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]

In [8]:
five_hot_dev = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  dev.label.to_numpy().reshape(-1, 1))

sentences_dev = [my_text_to_word_sequence(sentence) for sentence in dev['sentence']]

In [9]:
five_hot_test_org = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test_org.label.to_numpy().reshape(-1, 1))

sentences_test_org = [my_text_to_word_sequence(sentence) for sentence in test_org['sentence']]

In [10]:
five_hot_test = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test.label.to_numpy().reshape(-1, 1))

sentences_test = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

In [11]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1))

sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

In [12]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1))

sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

In [13]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_dev):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Il massimo è 92


In [14]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
    for j, word in enumerate(sentence):
        try:
            embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [15]:
embedded_devset = np.zeros(shape=(len(sentences_dev), max, 300))
for i, sentence in enumerate(sentences_dev):
    for j, word in enumerate(sentence):
        try:
            embedded_devset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [16]:
embedded_testset_org = np.zeros(shape=(len(sentences_test_org), max, 300))
for i, sentence in enumerate(sentences_test_org):
    for j, word in enumerate(sentence):
        try:
            embedded_testset_org[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [17]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
    for j, word in enumerate(sentence):
        try:
            embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [18]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
    for j, word in enumerate(sentence):
        try:
            embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [19]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
    for j, word in enumerate(sentence):
        try:
            embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

# Model

In [20]:
best_params = optuna.load_study(study_name="DDI",
                                storage="sqlite:///"+path+"ddi2013-type/optuna_ddi_studio_0.db").best_params

In [21]:
print(f'{best_params}')

{'batch_size': 89, 'dropout': 0.63, 'units': 81}


In [22]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [24]:
result = model.fit(embedded_trainset,
                   five_hot_train,
                   validation_data=(embedded_devset, five_hot_dev),
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                            patience=10,
                                                            restore_best_weights=True)])

Epoch 1/100
422/422 [==============================] - 73s 162ms/step - loss: 0.6776 - accuracy: 0.8363 - val_loss: 0.4712 - val_accuracy: 0.8621
Epoch 2/100
422/422 [==============================] - 73s 173ms/step - loss: 0.4837 - accuracy: 0.8471 - val_loss: 0.4492 - val_accuracy: 0.8657
Epoch 3/100
422/422 [==============================] - 73s 174ms/step - loss: 0.4415 - accuracy: 0.8462 - val_loss: 0.4209 - val_accuracy: 0.8642
Epoch 4/100
422/422 [==============================] - 74s 174ms/step - loss: 0.4113 - accuracy: 0.8505 - val_loss: 0.4470 - val_accuracy: 0.8548
Epoch 5/100
422/422 [==============================] - 74s 175ms/step - loss: 0.3907 - accuracy: 0.8515 - val_loss: 0.4214 - val_accuracy: 0.8673
Epoch 6/100
422/422 [==============================] - 74s 174ms/step - loss: 0.3732 - accuracy: 0.8559 - val_loss: 0.4134 - val_accuracy: 0.8690
Epoch 7/100
422/422 [==============================] - 74s 175ms/step - loss: 0.3594 - accuracy: 0.8564 - val_loss: 0.4316 -

In [31]:
model.save_weights('DDI1_005.h5')

# EVALUATION

In [25]:
def print_confusionMatrix_fscore(prediction, hot_encoding):
    cm = confusion_matrix(hot_encoding.argmax(axis=1), prediction.argmax(axis=1))
    fscore = f1_score(y_true=hot_encoding.argmax(axis=1),
                      y_pred=prediction.argmax(axis=1),
                      average=None)

    print('Confusion Matrix:\n\t\tDDI-false\tDDI-mechanism\tDDI-effect\tDDI-advise\tDDI-int')
    print(f'DDI-false\t{cm[0][0]}\t\t{cm[0][1]}\t\t{cm[0][2]}\t\t{cm[0][3]}\t\t{cm[0][4]}')
    print(f'DDI-mechanism\t{cm[1][0]}\t\t{cm[1][1]}\t\t{cm[1][2]}\t\t{cm[1][3]}\t\t{cm[1][4]}')
    print(f'DDI-effect\t{cm[2][0]}\t\t{cm[2][1]}\t\t{cm[2][2]}\t\t{cm[2][3]}\t\t{cm[2][4]}')
    print(f'DDI-advise\t{cm[3][0]}\t\t{cm[3][1]}\t\t{cm[3][2]}\t\t{cm[3][3]}\t\t{cm[3][4]}')
    print(f'DDI-int\t\t{cm[4][0]}\t\t{cm[4][1]}\t\t{cm[4][2]}\t\t{cm[4][3]}\t\t{cm[4][4]}')

    zipped_fscore = zip(['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int'], fscore)
    print('\n\nFSCORE:')
    for nm, val in zipped_fscore:
        print(f'{nm}: {val}')

In [26]:
from sklearn.metrics import confusion_matrix, f1_score

## DATASET ORIGINARIO

In [27]:
result_base=model.evaluate(embedded_trainset, five_hot_train, batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

422/422 [==============================] - 24s 56ms/step - loss: 0.2400 - accuracy: 0.9008
DATASET ORIGINARIO[0.24004089832305908, 0.9008467197418213]


In [28]:
pred = model.predict(embedded_trainset, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_train)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	31025		180		407		57		15
DDI-mechanism	1144		735		13		0		0
DDI-effect	1000		13		1404		5		2
DDI-advise	718		1		3		544		0
DDI-int		165		0		1		0		126


FSCORE:
DDI-false: 0.943927224047706
DDI-mechanism: 0.5210918114143921
DDI-effect: 0.6603951081843837
DDI-advise: 0.5811965811965812
DDI-int: 0.5793103448275861


## TESTSET

In [29]:
result_base=model.evaluate(embedded_testset, five_hot_test, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

130/130 [==============================] - 7s 56ms/step - loss: 0.5635 - accuracy: 0.8338
DATASET TEST[0.5635077953338623, 0.8337962031364441]


In [30]:
pred = model.predict(embedded_testset, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_test)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	9135		105		283		36		5
DDI-mechanism	491		104		8		1		0
DDI-effect	456		2		258		4		0
DDI-advise	329		2		2		109		0
DDI-int		160		2		29		0		1


FSCORE:
DDI-false: 0.9073752172833375
DDI-mechanism: 0.25396825396825395
DDI-effect: 0.396923076923077
DDI-advise: 0.36824324324324326
DDI-int: 0.0101010101010101


## TESTSET ORIGINARIO

In [32]:
result_base=model.evaluate(embedded_testset_org, five_hot_test_org, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

65/65 [==============================] - 4s 56ms/step - loss: 0.5422 - accuracy: 0.8349
DATASET TEST[0.5421619415283203, 0.8349245190620422]


In [33]:
pred = model.predict(embedded_testset_org, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_test_org)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	4553		52		157		18		2
DDI-mechanism	246		55		1		0		0
DDI-effect	214		0		144		2		0
DDI-advise	161		1		1		58		0
DDI-int		79		1		16		0		0


FSCORE:
DDI-false: 0.9074240159441953
DDI-mechanism: 0.267639902676399
DDI-effect: 0.42415316642120765
DDI-advise: 0.38795986622073586
DDI-int: 0.0


## DATASET SINONIMI

In [34]:
result_base=model.evaluate(embedded_sin, five_hot_sin, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 3s 49ms/step - loss: 0.5849 - accuracy: 0.8327
DATASET SINONIMI[0.58485347032547, 0.8326679468154907]


In [35]:
pred = model.predict(embedded_sin, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_sin)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	4582		53		126		18		3
DDI-mechanism	245		49		7		1		0
DDI-effect	242		2		114		2		0
DDI-advise	168		1		1		51		0
DDI-int		81		1		13		0		1


FSCORE:
DDI-false: 0.9073267326732674
DDI-mechanism: 0.24019607843137253
DDI-effect: 0.36714975845410625
DDI-advise: 0.348122866894198
DDI-int: 0.019999999999999997


## DATASET EMBEDDING

In [36]:
result_base=model.evaluate(embedded_emb, five_hot_emb, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 3s 42ms/step - loss: 0.6053 - accuracy: 0.8294
DATASET SINONIMI[0.6052672266960144, 0.8293699026107788]


In [37]:
pred = model.predict(embedded_emb, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_emb)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	4576		54		112		39		1
DDI-mechanism	253		48		1		0		0
DDI-effect	251		4		103		2		0
DDI-advise	167		0		3		51		0
DDI-int		80		0		16		0		0


FSCORE:
DDI-false: 0.9053318824809576
DDI-mechanism: 0.23529411764705882
DDI-effect: 0.34621848739495803
DDI-advise: 0.32587859424920124
DDI-int: 0.0
